In [55]:
import pandas as pd
import numpy as np

<div dir="rtl" style=" font-size: 20px;">
>>> استخراج فایل CSV داده‌ها
</div>

In [56]:
import requests
from datetime import datetime, timedelta

end_date = datetime.today().date()
start_date = end_date - timedelta(days=30)

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "csv",
    "starttime": str(start_date),
    "endtime": str(end_date),
    "minlatitude": 24,
    "maxlatitude": 46,
    "minlongitude": 123,
    "maxlongitude": 146,
    "minmagnitude": 1
}
response = requests.get(url, params=params)
with open("japan_earthquakes.csv", "w", encoding="utf-8") as f:
        f.write(response.text)


<div dir="rtl" style=" font-size: 20px;">
>>> خواندن فایل سیو شده در سیستم</div>

In [ ]:
df = pd.read_csv("japan_earthquakes.csv")
df

<div dir="rtl" style=" font-size: 20px;">
>>> حذف 3 ستون بدردنخور</div>

In [64]:
df = df.drop(columns=["status","locationSource", "magSource"])

<div dir="rtl" style=" font-size: 20px;">
>>> تبدیل داده‌های زمانی به تایم‌آبجکت ( اینجا میشد از متدهای خود پانداس هم استفاده کرد)
</div>

In [ ]:
import datetime

def date_objector(string):
    return datetime.datetime.strptime(string, "%Y-%m-%dT%H:%M:%S.%fZ")
df["time"] = df["time"].apply(date_objector)
df["updated"] = df["updated"].apply(date_objector)
# df["time"] = pd.to_datetime(df["time"])
# df["updated"] = pd.to_datetime(df["updated"])
df

<div dir="rtl" style=" font-size: 20px;">
>>> تعداد ردیف و ستون
</div>

In [ ]:
print("تعداد ردیف: ", df.shape[0])
print("تعداد ستون: ", df.shape[1])


<div dir="rtl" style=" font-size: 20px;">
>>> دیتاتایپ ستون‌ها
</div>

In [ ]:
df.dtypes

<div dir="rtl" style=" font-size: 20px;">
>>> تبدیل تمام مقادیر عددی به float
</div>


In [ ]:
num_cols = df.select_dtypes(include=["number"]).columns
df[num_cols] = df[num_cols].astype(float)
df

<div dir="rtl" style=" font-size: 20px;">
>>> بررسی وجود داده‌های null در ستون‌ها
</div>

In [ ]:
df.isnull().sum()

<div dir="rtl" style=" font-size: 20px;">
>>> ایجاد ستون نام ماه با استفاده از ستون تایم
</div>

In [ ]:
df["Month"] = df["time"].dt.month_name()
df

<div dir="rtl" style=" font-size: 20px;">
>>> ساخت ستون category
</div>

In [ ]:
def category(mag_str):
    mag = mag_str
    if mag < 4:
        return "Weak" 
    elif 4 <= mag <= 6:
        return "Moderate"
    elif mag > 6:
        return "Strong" 
    
df["Category"] = df["mag"].apply(category)
df

<div dir="rtl" style=" font-size: 20px;">
>>> ساخت ستون region با تابع place به روش زیر:</div>

In [ ]:
def place(place):
    a = place.split()
    if "of" in a:
        #return " ".join(a[2:]) 
        return " ".join(a[4:])
    else:
        return place
df["region"] = df["place"].apply(place)
df["region"].value_counts()

<div dir="rtl" style=" font-size: 20px;">
>>> گروه‌بندی داده‌ها به نام ماه و category و محاسبه شاخص‌های تعداد و میانگین براساس این دو تیتر</div>

In [ ]:
monthly_category = df.groupby(['Month', 'Category']).agg({'mag': ['count', 'mean'],'depth': 'mean'}).round(2)

print(monthly_category)

<div dir="rtl" style=" font-size: 20px;">
>>> گروه‌بندی براساس نام ناحیه زلزله و محاسبه شاخص‌های زیر:</div>

In [ ]:
region_stats = df.groupby('region').agg({'mag': ['count', 'mean', 'max'],'depth': ['mean', 'max'],'latitude': 'mean','longitude': 'mean'}).round(2)

print(region_stats)

<div dir="rtl" style=" font-size: 20px;">
>>> تعداد زلزله براساس ناحیه</div>

In [ ]:
region_counts = df.groupby('region').size().sort_values(ascending=False)

print(region_counts)

<div dir="rtl" style=" font-size: 20px;">
>>> نمودار میله‌ای پراکندگی زلزله‌ها براساس ناحیه
</div>

In [ ]:
region_counts.plot(kind='bar', color='skyblue', edgecolor='black')


<div dir="rtl" style=" font-size: 20px;">
>>> محاسبه فاصله نقاط زلزله با توکیو
</div>

In [ ]:

tokyo_lat, tokyo_lon = 35.6895, 139.6917

df['dist_to_tokyo'] = np.sqrt(
    (df['latitude'] - tokyo_lat)**2 + 
    (df['longitude'] - tokyo_lon)**2
)

print(df[["region",'latitude', 'longitude', 'dist_to_tokyo']])

<div dir="rtl" style=" font-size: 20px;">
>>> محاسبه‌ی شاخص‌های موردنظر با نامپای:</div>

In [ ]:
magnitudes = df['mag'].to_numpy()
distances = df['dist_to_tokyo'].to_numpy()

mean_mag = np.mean(magnitudes)
std_mag = np.std(magnitudes)
percentile_90 = np.percentile(magnitudes, 90)
mean_distance = np.mean(distances)

print("میانگین زلزله ها: ", mean_mag)
print("انحراف معیار: ", std_mag)
print("صدک 90ام: ", percentile_90)
print("میانگین فاصله از توکیو: ", mean_distance)